In [1]:
import findspark
findspark.init()

In [2]:
import json
from pyspark import sql
from pyspark.sql import functions as sfunc
import pandas as pd
pd.set_option('display.max_columns', 50)

In [3]:
with open('yelp_categories.json') as f:
    yelp_categories = json.load(f)
distinct_categories = list(set(yelp_categories.values()))

In [4]:
sc = sql.SparkSession.builder.appName('test').master('local[2]').enableHiveSupport().getOrCreate()

In [5]:
HDFS_HOME = "/home/ajay/big_data/hadoop-3.3.6/hdfs/data/datanode"

In [6]:
df = sc.read.json("C:/Users/ajayd/Home/VM/data/yelp_academic_dataset_business.json", mode = "DROPMALFORMED", dropFieldIfAllNull = True)

In [7]:
df.limit(3).toPandas()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2","(None, None, None, None, None, None, None, Non...",Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,None,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA
1,87 Grasso Plaza Shopping Center,"(None, None, None, None, None, None, None, Non...",mpf3x-BjTdTEA3yCZrAYPw,"Shipping Centers, Local Services, Notaries, Ma...",Affton,"(8:0-18:30, 0:0-0:0, 8:0-14:0, None, 8:0-18:30...",1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO
2,5255 E Broadway Blvd,"(None, None, None, None, None, None, None, Tru...",tUFrWirKiKi_TAnsVWINQQ,"Department Stores, Shopping, Fashion, Home & G...",Tucson,"(8:0-23:0, 8:0-22:0, 8:0-23:0, 8:0-22:0, 8:0-2...",0,32.223236,-110.880452,Target,85711,22,3.5,AZ


In [8]:
df = df.filter(df.is_open == 1)

In [9]:
df = df.withColumn('categories', sfunc.split(df.categories, ", "))

In [10]:
@sfunc.pandas_udf(sql.types.ArrayType(sql.types.StringType()))
def map_categories(srs: pd.Series) -> pd.Series:
    return srs.apply(lambda x: [] if type(x) == type(None) else [yelp_categories.get(i) for i in x])

In [11]:
df = df.withColumn('categories', sfunc.array_distinct(map_categories(df.categories)))

In [12]:
print(df.filter(sfunc.size(df.categories) == 0).count())
df = df.filter(sfunc.size(df.categories) != 0)
df.select(sfunc.size(df.categories)).describe().show()

95
+-------+------------------+
|summary|  size(categories)|
+-------+------------------+
|  count|            119603|
|   mean| 1.541065023452589|
| stddev|0.7616206701463434|
|    min|                 1|
|    max|                 9|
+-------+------------------+



In [13]:
dummy_df = df.withColumn("category", sfunc.explode(df.categories)).groupBy(df.business_id).pivot('category').agg(sfunc.count('*')).fillna(0)

In [14]:
df = df.join(dummy_df, on = 'business_id')

In [15]:
df = df.select('business_id', 'attributes.*', 'city', 'hours', 'latitude', 'longitude', 'name', 
               'review_count', 'stars', 'state', *distinct_categories)

In [16]:
less_populated_features = ['AgesAllowed', 'BYOB', 'BYOBCorkage','CoatCheck','Corkage','DietaryRestrictions','DogsAllowed','GoodForDancing', 
                             'Open24Hours', 'RestaurantsCounterService', 'AcceptsInsurance']
df = df.drop(*less_populated_features)
df.columns

['business_id',
 'Alcohol',
 'Ambience',
 'BestNights',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'ByAppointmentOnly',
 'Caters',
 'DriveThru',
 'GoodForKids',
 'GoodForMeal',
 'HairSpecializesIn',
 'HappyHour',
 'HasTV',
 'Music',
 'NoiseLevel',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi',
 'city',
 'hours',
 'latitude',
 'longitude',
 'name',
 'review_count',
 'stars',
 'state',
 'Shopping',
 'Hotels & Travel',
 'Pets',
 'Active Life',
 'Nightlife',
 'Automotive',
 'Home Services',
 'Food',
 'Arts & Entertainment',
 'Public Services & Government',
 'Local Services',
 'Health & Medical',
 'Professional Services',
 'Beauty & Spas',
 'Financial Services',
 'Event Planning & Services',
 'Mass Media',
 'Religious Organizations',
 'Educatio

In [17]:
@sfunc.pandas_udf(sql.types.StringType())
def eval_u_strings(srs: pd.Series) -> pd.Series:
    return srs.map(lambda x: x.strip("u'").strip('u"') if x and x.strip("u'").lower() != 'none' else "")

df = df.withColumns({
                'Alcohol' : eval_u_strings(df.Alcohol),
                'NoiseLevel' : eval_u_strings(df.NoiseLevel),
                'RestaurantsAttire' : eval_u_strings(df.RestaurantsAttire),
                'WiFi' : eval_u_strings(df.WiFi)
            })

In [18]:
df = df.fillna('unknown', subset = ['RestaurantsPriceRange2']).replace({"1" : "inexpensive", 
                              "2" : "moderately priced", 
                              "3" : "expensive", 
                              "4" : "very expensive"}, subset = ['RestaurantsPriceRange2'])

boolean_columns = ['BikeParking', 'ByAppointmentOnly', 'Caters', 'GoodForKids', 'HappyHour', 'HasTV', 'OutdoorSeating',
            'RestaurantsDelivery', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'RestaurantsTableService',
            'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'BusinessAcceptsBitcoin','BusinessAcceptsCreditCards',
            'DriveThru'
                  ]
df = df.na.fill("unknown", subset = boolean_columns).replace({"None" : "unknown"}, subset = boolean_columns)

In [19]:
dict_like_cols = ["Ambience", "BusinessParking", "BestNights", "GoodForMeal", "HairSpecializesIn", "Music"]

def convert_str_to_json(srs: pd.Series) -> pd.Series:
    srs = srs.fillna("{}").str.replace("u'", "'").str.replace('u"', "'").str.replace("'", '"').str.replace("None", "False").apply(eval)
    return pd.DataFrame(srs.tolist(), index = srs.index).fillna(-1).replace({True :1, False : 0})

In [20]:
df = df.na.fill("{}", subset = dict_like_cols
               ).replace({"None" : "{}"}, subset = dict_like_cols
                        ).withColumns({i : sfunc.pandas_udf(convert_str_to_json, 
                                                            sql.types.StructType([
                                                                sql.types.StructField(key, sql.types.IntegerType(), True) 
                                        for key in eval(df.select(i).filter(sfunc.length(i) > 5).first().__getattr__(i))])
                                                           )(sfunc.col(i))
                   for i in dict_like_cols
        })

In [21]:
col_types = df.dtypes

In [26]:
df.select(*[i[0] for i in col_types if not i[1].startswith('struct')], *[i[0] + '.*' for i in col_types if i[1].startswith('struct')]).limit(20).toPandas()

,business_id,Alcohol,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,ByAppointmentOnly,Caters,DriveThru,GoodForKids,HappyHour,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,city,latitude,...,dinner,brunch,breakfast,straightperms,coloring,extensions,africanamerican,curly,kids,perms,asian,dj,background_music,no_music,jukebox,live,video,karaoke,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,---kPU91CF4Lq2-WlRu9Lw,,True,False,True,unknown,True,unknown,unknown,unknown,False,,True,,False,unknown,unknown,False,True,True,unknown,True,no,New Port Richey,28.217288,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,11:0-22:0,0:0-0:0,11:0-22:0,None,11:0-21:0,11:0-21:0,11:0-21:0
1,--9osgUCSDUWUkoTLdvYhQ,,True,unknown,True,True,unknown,unknown,True,unknown,unknown,,unknown,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,,Tucson,32.253677,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,11:0-22:0,11:0-22:0,16:0-22:0,15:0-20:0,11:0-22:0,11:0-22:0,11:0-22:0
2,--FcbSxK1AoEtEAxOgBaCw,,unknown,unknown,True,False,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,,Riverview,27.914364,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,None,None,None,None,None,None,None
3,--LC8cIrALInl2vyo701tg,,True,unknown,True,True,unknown,unknown,True,unknown,unknown,,unknown,,unknown,unknown,moderately priced,unknown,unknown,unknown,unknown,unknown,,Tampa,28.065886,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,9:0-20:0,None,8:0-16:0,None,9:0-20:0,9:0-17:0,9:0-20:0
4,--OS_I7dnABrXvRCCuWOGQ,,unknown,unknown,True,unknown,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,,Philadelphia,40.028160,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,7:30-17:0,7:30-17:0,8:0-12:0,None,7:30-17:0,7:30-17:0,7:30-17:0
5,--epgcb7xHGuJ-4PUeSLAw,,True,unknown,True,unknown,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,inexpensive,unknown,unknown,True,unknown,unknown,,Willow Grove,40.145054,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0
6,--gJkxbsiSIwsQKbiwm_Ng,,True,False,True,False,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,True,free,Tampa,27.945122,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,8:0-20:0,8:0-20:0,8:0-20:0,12:0-17:0,8:0-20:0,8:0-20:0,8:0-20:0
7,--hF_3v1JmU9nlu4zfXJ8Q,beer_and_wine,True,unknown,True,unknown,True,unknown,unknown,unknown,unknown,,True,,True,unknown,unknown,unknown,unknown,True,unknown,True,free,Indianapolis,39.767888,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,11:0-18:0,0:0-0:0,None,None,11:0-16:0,11:0-16:0,11:0-16:0
8,--rS-rnOIZxoiDA8yctWpQ,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,moderately priced,unknown,unknown,unknown,unknown,unknown,,Tampa,27.943291,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0
9,--seHXaJfDtWbmFSxkzaRg,,unknown,unknown,unknown,True,unknown,unknown,unknown,unknown,unknown,,unknown,,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,,Collingswood,39.910771,...,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,None,9:30-17:30,None,None,8:0-19:30,8:30-19:30,8:30-12:30
